# 기계 번역 모델 평가

이번 실습에서는 기계 번역 모델을 평가하는 방법에 대해 살펴보겠습니다.

기계 번역이란 하나의 언어로 적혀져있는 문장을 다른 언어의 문장으로 번역하는 분야를 뜻합니다.

룰에 따라 번역을 하는 단순한 기계 번역부터 딥러닝 모델을 이용한 최신 모델까지 다양하게 있습니다.

지난 실습에서는 이러한 기계 번역을 수행하는 딥러닝 모델 중 하나인 RNN 기반의 seq2seq 모델의 학습에 대해 살펴보았습니다.

이번 실습에서는 Transformer 기반에 이미 학습이 완료된 모델을 통해 그 결과를 살펴보겠습니다.

그리고 해당 모델의 성능이 얼마나 뛰어난지를 살펴보는 모델 평가에 대해 진행하겠습니다.

기계 번역 모델의 평가는 기존에 살펴보았던 여러 모델들 평가와는 조금 다릅니다.

기존에 보았던 문제들은 classification입니다. 

즉, 입력으로 들어온 데이터를 보고 미리 정한 클래스 중 가장 적합한 클래스를 선택하는 것입니다.

대표적으로 감정 분석의 경우 주어진 텍스트를 보고 긍정, 부정의 클래스를 맞추는 것이지요.

이런 경우 그 모델의 평가는 대체로 정확도(accuracy)로 계산합니다.

모델이 내놓은 결과인 클래스와 실제 데이터의 결과인 클래스를 비교하여 얼마나 맞추었는지를 살펴보는 것이지요.



하지만 기계 번역 모델의 경우 모델과 데이터의 결과 모두 단어의 조합인 문장입니다.

그렇기에 단순히 정확도로 판단하기에는 어렵습니다.

아래 예시를 살펴보겠습니다.

- 입력 문장: 오늘 저는 여러분들과 만나게 되어 반갑습니다
- 데이터 내 영어 번역 문장: Today I am glad to meet you
- 모델의 번역 문장: It is a pleasure to meet you today

두 번째 문장과 세 번째 문장 모두 첫 번째 문장에 적합한 번역 문장입니다.

하지만 단순히 두 문장이 정확하게 일치하는지를 살펴보게 된다면 첫 번째 단어가 다르기에 0점을 주게 되는 문제가 발생합니다.

이런 문제를 해결하고자 여러 방법이 제안되었습니다.

그 중에서 이번 실습에서는 ROUGE(Recall-Oriented Understudy for Gisting Evaluation)에 대해 살펴보겠습니다.

## 학습된 기계 번역 모델 사용

ROUGE에 대해 살펴보기에 앞서 먼저 학습된 번역 모델을 살펴보겠습니다.

여러 기계 번역 모델이 있지만 그 중에서 한국어-영어 번역 모델을 하나 가져오겠습니다.

출처: https://huggingface.co/Helsinki-NLP/opus-mt-ko-en

코드 실행에 필요한 라이브러리를 설치하고 로드합니다.

In [1]:
!pip install torch==1.7.0
!pip install transformers==3.5.1

     |████████████████████████████████| 776.7 MB 4.4 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.0 which is incompatible.
     |████████████████████████████████| 1.3 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 
     |████████████████████████████████| 2.9 MB 39.7 MB/s 
     |████████████████████████████████| 1.1 MB 45.9 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

텍스트를 나눌 tokenizer와 학습 모델을 불러옵니다.

In [3]:
ko_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ko-en")

mt_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ko-en")

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/842k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/813k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/312M [00:00<?, ?B/s]

불러온 모델을 가지고 번역을 진행해보겠습니다.

In [4]:
text1 = "대한민국은 민주공화국이다"
text2 = "대한민국의 주권은 국민에게 있고 모든 권력은 국민으로부터 나온다"
tokenized_text = ko_tokenizer.prepare_seq2seq_batch([text1, text2])

translation = mt_model.generate(**tokenized_text)
translated_text = ko_tokenizer.batch_decode(translation, skip_special_tokens=True)

print(translated_text)

['South Korea is the Democratic Republic.', "South Korea's sovereignty comes from the people, and all power comes from the people."]


결과가 나름 깔끔하게 잘 나오는 것을 알 수 있습니다.

## ROUGE

이제 이렇게 학습된 모델을 평가하도록 하겠습니다.

평가 방법은 ROUGE입니다.

ROUGE는 원래 텍스트 요약에 제안된 평가 방법입니다. 

요약은 긴 글을 짧은 문장으로 바꾸는 문제로 데이터에는 실제 정답 요약문이 있습니다. 그리고 모델은 긴 글을 입력으로 받아 짧은 요약문을 만듭니다. 그러면 ROUGE는 정답 요약문과 생성된 요약문을 비교하여 그 유사도를 계산합니다.

이러한 접근법은 기계 번역에서도 유사합니다.

기계 번역 역시 실제 정답 문장이 있고 모델이 번역한 문장이 있습니다. 그리고 이 둘이 얼마나 유사한지를 계산하면 됩니다.

ROUGE는 이처럼 두 문장의 유사도를 계산하는 방법이라고 보시면 됩니다.

ROUGE 알고리즘은 기본적으로 두 문장의 단어가 겹치는 부분을 계산합니다.

- 입력 문장: 오늘 저는 여러분들과 만나게 되어 반갑습니다
- 정답 문장: Today I am glad to meet you
- 번역 문장: It is a pleasure to meet you today

위의 예시에서 정답 문장과 번역 문장을 비교하면 우리는 두 문장 내 단어가 얼마나 겹치는지를 살펴보면서 유사도를 계산할 수 있습니다.

`to`, `meet`, `you` 단어 3개가 두 문장 내에 나타나는 것을 알 수 있습니다.

그리고 번역 문장에서 단어의 개수는 총 8개입니다.

그렇다면 8개 단어를 모델이 만들었고 그 중 3개가 정답 문장에 있는 것이니 $3/8=0.375$의 수치로 모델 성능을 평가할 수 있습니다.

이렇게 계산한 값을 `precision`이라고 합니다.

하지만 이것만이 중요할까요?

반대로 정답 문장에 있는 단어들을 모델이 얼마나 찾아내어 이를 생성하였는지 중요합니다.

즉, 정답 문장 내 7개 단어들 중 모델은 총 3개를 만들었기에 $3/7=0.428$의 수치 역시 모델 성능을 평가할 수 있습니다.

이렇게 계산한 값을 `recall`이라고 합니다.

이렇게 `precision`과 `recall`은 각각 ROUGE를 통해 계산한 모델의 성능 수치입니다.

이 둘을 모아서 하나의 값으로 보고 싶을 수 있습니다.

그럴 때 우리는 이 둘의 조화 평균값을 구합니다.

$$F = \frac{2 \times P \times R}{P + R}$$

$P$: `precision`, $R$: `recall`, $F$: `F score`

이렇게 계산한 값을 `F score` 이라고 부릅니다.

Tip) 가끔 `precision`과 `recall` 모두 0이 되는 경우가 있습니다. 즉, 두 문장에 아무것도 겹치는 단어가 없는 경우입니다. 이럴 때 `F score`를 계산해버리면 분모가 0이 되어 문제가 됩니다. 이를 해결하기 위해 아주 작은 값을 분모에 더하면 이런 문제를 방지할 수 있습니다.

ROUGE는 이처럼 겹치는 단어만을 보고 모델의 성능을 평가하지 않습니다.

아래와 같은 예시를 보겠습니다.

- 입력 문장: 오늘 저는 여러분들과 만나게 되어 반갑습니다
- 정답 문장: Today I am glad to meet you
- 번역 문장1: It is a pleasure to meet you today
- 번역 문장2: It a to today meet pleasure is you 

번역 문장 1과 2는 단어는 같지만 그 순서가 다릅니다.

그리고 문장 2는 좋지 못한 번역 문장임을 알 수 있습니다.

하지만 단어만으로 ROUGE를 계산하게 된다면 번역 문장 1과 2는 동일한 값을 가지는 문제가 있습니다.

이를 해결하기 위해서는 어떻게 해야할까요?

단어의 순서를 고려하는 방법 중 하나가 바로 n-gram입니다.

n-gram은 주어진 n개의 단어를 하나의 단위로 묶어서 살펴보는 방법입니다.

n이 1인 경우는 unigram이라 불리어 단어 하나씩 보는 경우를 뜻합니다.

n이 2인 경우는 bigram이라 불리어 연속된 두 단어를 하나로 묶어 살펴봅니다.

예를 들어 정답 문장의 bigram을 구하면 아래와 같습니다.

- Today I
- I am
- am glad
- glad to
- to meet
- meet you

번역 문장1의 bigram을 구하면 아래와 같습니다.
- It is
- is a
- a pleasure
- pleasure to
- to meet
- meet you
- you today

번역 문장2의 bigram을 구하면 아래와 같습니다.
- It a
- a to
- to today
- today meet
- meet pleasure
- pleasure is
- is you

이렇게 bigram으로 token을 만든 후 비교해보면 정답 문장과 번역 문장1은 2개가 겹칩니다.

하지만 정답 문장과 번역 문장2는 하나도 겹치는 bigram token이 없습니다. 즉, ROUGE precision, recall, F score 모두 0의 값입니다.

이를 통해 번역 문장1이 번역 문장2보다는 좋은 문장임을 알 수 있습니다.

이번 실습에서는 ROUGE를 직접 구현하도록 하겠습니다.

ROUGE에는 여러 기법이 있지만 그 중 n-gram을 사용한 방법을 소개하였고 이를 구현하겠습니다.

### unigram

- 문제 1. unigram을 만드는 함수를 작성해주세요.
  - 문장(sentence)와 토크나이저(tokenizer)를 입력으로 받아 토크나이저를 이용하여 unigram을 만들어주세요.
  - 결과는 Python의 set의 형태로 반환해주세요.
  - 문장 내 중복으로 나타나는 단어는 한 번 나타난 것으로 취급해주세요.
  - `ngram` 함수에서 `n`이 1인 경우와 동일해야합니다. 하지만 해당 함수를 이 함수 내에서 사용하지 마세요.



In [5]:
def unigram(sentence, tokenizer):
    results_unigram = set()
    # <ToDo>: sentence에서 unigram을 만드세요.
    results_unigram = tokenizer(sentence)
    results_unigram = set(results_unigram)
    results_unigram = list(results_unigram)

    return results_unigram

#nigram = nltk.ngrams('asjbfajskdbfkh asdfbhasdbfika askjdhfbajh basdhfbh djdj',n=1)
#print(set(nigram))
#token = nltk.word_tokenize
#unigram('asdjufhbasuj akhsbjbasdf asdjkfbakshjdbf khasdfkjashdfkjashd kjfhask', token)


### bigram

- 문제 2. bigram을 만드는 함수를 작성해주세요.
  - 문장(sentence)와 토크나이저(tokenizer)를 입력으로 받아 토크나이저를 이용하여 bigram을 만들어주세요.
  - 결과는 Python의 set의 형태로 반환해주세요.
  - 중복으로 나타나는 bigram token은 한 번 나타난 것으로 취급해주세요.
  - `ngram` 함수에서 `n`이 2인 경우와 동일해야합니다. 하지만 해당 함수를 이 함수 내에서 사용하지 마세요.



In [6]:
def bigram(sentence, tokenizer):
    results_bigram = set()
    # <ToDo>: sentence에서 bigram을 만드세요.
    results_bigram = tokenizer(sentence)
    results_bigram = set(results_bigram)
    results_bigram = list(results_bigram)

    return results_bigram

### n-gram

- 문제 3. n-gram을 만드는 함수를 작성해주세요.
  - 문장(sentence)와 토크나이저(tokenizer)를 입력으로 받아 토크나이저를 이용하여 n-gram을 만들어주세요.
  - 결과는 Python의 set의 형태로 반환해주세요.
  - 중복으로 나타나는 n-gram token은 한 번 나타난 것으로 취급해주세요.
  - n이 1일 때와 2일 때는 각각 `unigram`과 `bigram` 함수와 동일한 결과를 출력해야합니다.



In [7]:
def ngram(sentence, tokenizer, n):
    results_ngram = set()
    # <ToDo>: sentence에서 ngram을 만드세요.
    results_ngram = tokenizer(sentence)
    results_ngram = set(results_ngram)
    results_ngram = list(results_ngram)

    return results_ngram

### ROUGE

- 문제 4. ROUGE를 계산하는 함수를 작성해주세요.
  - 정답 문장(original_sentence)와 번역 문장(translated_sentence)의 차이를 ROUGE로 계산해주세요.
  - 토크나이저(tokenizer)를 통해 단어 토큰을 만들고 n의 크기에 따라 n-gram 토큰을 만드세요.
  - 결과는 ROUGE의 precision, recall, F score 세 개를 반환해주세요.
  - 가끔 입력 문장이 빈 문장(길이가 0인 문장)일 수 있습니다. 이 때 계산 시 분모가 0이 되는 값이 생길 수 있습니다. 이런 경우 그 값을 0으로 만들어주세요.
  - F score 계산 시 분모가 0이 되는 경우를 주의해주세요. (힌트: small_num 값 사용)



In [18]:
def rouge(original_sentence, translated_sentence, n, tokenizer, small_num=1e-8):
    # 입력 문장들을 전부 소문자로 바꿉니다.
    original_sentence = original_sentence.lower()
    translated_sentence = translated_sentence.lower()

    # n의 크기에 따라 함수를 호출합니다.
    if n <= 0:
        print("n은 0보다 커야합니다.")
        return None
    elif 1 == n:
        original_grams = unigram(original_sentence, tokenizer)
        translated_grams = unigram(translated_sentence, tokenizer)
    elif 2 == n:
        original_grams = bigram(original_sentence, tokenizer)
        translated_grams = bigram(translated_sentence, tokenizer)
    else:
        original_grams = ngram(original_sentence, tokenizer, n)
        translated_grams = ngram(translated_sentence, tokenizer, n)

    # 찾아낸 gram들의 개수를 셉니다.
    original_count = len(original_grams)
    translated_count = len(translated_grams)

    # <ToDo>: 중첩되는 ngrams을 구하세요.
    # overlapping_ngrams = None
    #overlapping_ngrams = [i for i in original_grams if i in translated_grams]
    #print(overlapping_ngrams)
    overlapping_ngrams = set(original_grams) & set(translated_grams)
    overlapping_count = len(overlapping_ngrams)

    # <ToDo>: precision을 구하세요.
    # precision = None     # TP / (FP + TP)
    precision = original_count / (translated_count + original_count)

    # <ToDo>: recall을 구하세요.
    # recall = None     # TP / (FN + TP)
    recall = original_count / (overlapping_count + original_count)

    # <ToDo>: F score를 구하세요.
    # f_score = None     # f_score = 2*p*r / (p+r)
    f_score = 2 * precision * recall / (precision + recall) + small_num

    return precision, recall, f_score


### 실행 코드

In [19]:
# tokenizer는 nltk를 사용합니다.
en_tokenizer = nltk.word_tokenize

# 예시 문장
original_sen = "Today I am glad to meet you"
translated_sen1 = "It is a pleasure to meet you today"
translated_sen2 = "It a to today meet pleasure is you"

# 정답 문장과 번역 문장1을 ROUGE로 비교합니다.
print("Original: {}".format(original_sen))
print("Translated: {}".format(translated_sen1))

p, r, f = rouge(original_sen, translated_sen1, 1, en_tokenizer)
print("ROUGE unigram: Precision: {:.4f}, Recall: {:.4f}, F score:{:.4f}".format(p, r, f))

p, r, f = rouge(original_sen, translated_sen1, 2, en_tokenizer)
print("ROUGE bigram: Precision: {:.4f}, Recall: {:.4f}, F score:{:.4f}".format(p, r, f))

p, r, f = rouge(original_sen, translated_sen1, 3, en_tokenizer)
print("ROUGE trigram(n=3): Precision: {:.4f}, Recall: {:.4f}, F score:{:.4f}".format(p, r, f))
print()

# 정답 문장과 번역 문장2를 ROUGE로 비교합니다.
print("Original: {}".format(original_sen))
print("Translated: {}".format(translated_sen2))

p, r, f = rouge(original_sen, translated_sen2, 1, en_tokenizer)
print("ROUGE unigram: Precision: {:.4f}, Recall: {:.4f}, F score:{:.4f}".format(p, r, f))

p, r, f = rouge(original_sen, translated_sen2, 2, en_tokenizer)
print("ROUGE bigram: Precision: {:.4f}, Recall: {:.4f}, F score:{:.4f}".format(p, r, f))

p, r, f = rouge(original_sen, translated_sen2, 3, en_tokenizer)
print("ROUGE trigram(n=3): Precision: {:.4f}, Recall: {:.4f}, F score:{:.4f}".format(p, r, f))


Original: Today I am glad to meet you
Translated: It is a pleasure to meet you today
ROUGE unigram: Precision: 0.4667, Recall: 0.6364, F score:0.5385
ROUGE bigram: Precision: 0.4667, Recall: 0.6364, F score:0.5385
ROUGE trigram(n=3): Precision: 0.4667, Recall: 0.6364, F score:0.5385

Original: Today I am glad to meet you
Translated: It a to today meet pleasure is you
ROUGE unigram: Precision: 0.4667, Recall: 0.6364, F score:0.5385
ROUGE bigram: Precision: 0.4667, Recall: 0.6364, F score:0.5385
ROUGE trigram(n=3): Precision: 0.4667, Recall: 0.6364, F score:0.5385


ROUGE에도 한계점이 있다는 것을 알 수 있습니다. 대표적으로 'pleasure'와 'glad'는 비슷한 뜻을 지니고 있지만 ROUGE는 이 차이를 무조건 다르다고만 얘기하고 있기 때문입니다. 이러한 단점을 극복하고자 여러 방법이 제안되고 있습니다. 이처럼 NLP 모델의 평가는 현재 많은 연구가 활발하게 이루어지고 있는 연구 분야입니다.